In [12]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog


# Open a file selection dialog and get the selected file's path
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

# Load the dataframe
df = pd.read_excel(file_path, sheet_name="GISLeaseUpdateResults")
df

,Internal ID,Lease Type,Lease Subtype,Transaction Number,Lessee(s),Legacy Lease Number,Start Date (Letter Merge),End Date (Letter Merge),Related Asset,Name,Acreage,Legal Description,Lease Terms (Years),Administrator,District,Meridian,Township,Range,Section#,Lease Status
0,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,70.00,E2,15.0,Rachel Turner,North Central,6,5S,59W,10,Active
1,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002094,6-5S-59W-36-Surface-Arapahoe-School,640.00,ALL,15.0,Rachel Turner,North Central,6,5S,59W,36,Active
2,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,250.00,E2,15.0,Rachel Turner,North Central,6,5S,59W,10,Active
3,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004437,6-15S-62W-1-Surface-El Paso-School,637.12,"LTS 1-2, S2NE, SE, W2",25.0,Justin Osborne,South Central,6,15S,62W,1,Active
4,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004635,6-16S-62W-1-Surface-El Paso-School,609.18,"LTS 1-4, S2N2, S2",25.0,Justin Osborne,South Central,6,16S,62W,1,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37535,1227197,Temporary Access Permit,NaN,TAP-115160,NOBLE ENERGY INC,NaN,NaT,2023-12-31,FAM001330,6-2N-64W-16-Surface-Weld-School,640.00,ALL,NaN,Rachel Turner,North Central,6,2N,64W,16,Active
37536,1237565,Oil and Gas,Subsurface Easement,OG-115180,"EXTRACTION OIL & GAS, INC",NaN,NaT,NaT,FAM001248,6-1S-66W-36-Surface-Adams-School,400.00,"W2, W2SE",NaN,Catie Stitt,North Central,6,1S,66W,36,Active
37537,1238980,Temporary Access Permit,NaN,TAP-115182,ALTURA LAND CONSULTANTS,NaN,NaT,2023-03-31,FAM002310,6-6N-91W-14-Surface-Moffat-School,408.91,"NWNE, S2NE, PT SENW, PT SENW, PT NWSW, E2SWSW,...",NaN,Jerod Smith,Northwest,6,6N,91W,14,Active
37538,1238980,Temporary Access Permit,NaN,TAP-115182,ALTURA LAND CONSULTANTS,NaN,NaT,2023-03-31,FAM002309,6-6N-91W-13-Surface-Moffat-School,318.45,LOTS 9-16 (S2),NaN,Jerod Smith,Northwest,6,6N,91W,13,Active
